# KS-2: Motion → Demand Test

**Goal & Pass Criterion**: In a small lattice, demonstrate that "moving" patterns create higher demand than stationary ones. D_moving > D_stationary in active regions.

This notebook tests the hypothesis that motion/transport in quantum systems creates higher computational demand than stationary patterns.

## Setup

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from capacity_time.circuits import (
    create_standard_lattice, create_comparison_schedules, 
    evolve_circuit_schedule, measure_local_expectations, plot_lattice_dynamics
)
from capacity_time.demand_capacity import (
    create_default_demand_estimator, analyze_demand_vs_motion, 
    compute_demand_profile, plot_demand_capacity_analysis
)
from plotting import setup_figure, plot_heatmap, plot_line_comparison

# Configure plotting
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

## Theory Background

The core hypothesis is that quantum systems with **moving** patterns require higher computational "demand" to maintain than **stationary** patterns because:

1. Moving patterns create spatially varying entanglement
2. Information transport requires more "work" than static storage  
3. Local Hamiltonian variance increases with motion

We test this by comparing demand D estimated via generator variance:
$$D \approx 4 \frac{\text{Var}(H_{\text{local}})}{\hbar^2}$$

for moving vs stationary evolution schedules.

## Setup Lattice and Schedules

In [ ]:
# Create 6-qubit lattice
lattice = create_standard_lattice(n_qubits=6)
print(f"Created {lattice.n_qubits}-qubit lattice with {lattice.coupling_type} coupling")

# Create comparison schedules
schedules = create_comparison_schedules(
    n_qubits=6,
    n_steps=12, 
    amplitude=0.3
)

print(f"\nSchedules created:")
for name, schedule in schedules.items():
    print(f"- {name}: {len(schedule)} steps")
    
# Create demand estimator
demand_estimator = create_default_demand_estimator()
print(f"\nUsing demand estimation method: {demand_estimator.method}")

## Visualize Evolution Schedules

In [ ]:
# Plot the angle schedules to visualize patterns
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (name, schedule) in enumerate(schedules.items()):
    schedule_array = np.array(schedule)
    
    im = axes[i].imshow(schedule_array.T, aspect='auto', origin='lower', 
                       cmap='viridis', interpolation='nearest')
    axes[i].set_title(f'{name} Schedule')
    axes[i].set_xlabel('Time Step')
    axes[i].set_ylabel('Qubit Index')
    plt.colorbar(im, ax=axes[i], label='Rotation Angle (rad)')

plt.suptitle('KS-2: Evolution Schedule Comparison')
plt.tight_layout()
plt.savefig('figures/KS2_schedules.png', dpi=150, bbox_inches='tight')
plt.show()

print("Schedule patterns:")
print("- Moving: Localized rotation translates across lattice")
print("- Stationary: Localized rotation stays at center")
print("- Uniform: Equal rotation on all qubits (control)")

## Run Evolution Simulations

In [ ]:
# Run evolution for each schedule
print("Running quantum lattice evolution simulations...")

evolution_results = {}
initial_pattern = 'superposition'  # Start with |+⟩^⊗6

for name, schedule in schedules.items():
    print(f"  Evolving {name} pattern...")
    
    states = evolve_circuit_schedule(lattice, initial_pattern, schedule)
    expectations = measure_local_expectations(states, ['X', 'Z'])
    
    evolution_results[name] = {
        'states': states,
        'expectations': expectations,
        'schedule': schedule
    }
    
    print(f"    → Evolved for {len(states)} time steps")

print(f"\nCompleted {len(evolution_results)} evolution simulations")

## Visualize Lattice Dynamics

In [ ]:
# Plot space-time evolution of expectation values
for name, results in evolution_results.items():
    print(f"\nVisualizing {name} dynamics:")
    
    plot_lattice_dynamics(
        results['expectations'],
        title=f"KS-2: {name} Pattern Dynamics",
        save_path=f'figures/KS2_{name.lower()}_dynamics.png'
    )

## Compute and Compare Demand Profiles

In [ ]:
# Compute demand profiles for moving vs stationary
print("Computing demand profiles...")

moving_states = evolution_results['moving']['states']
stationary_states = evolution_results['stationary']['states']

# Run KS-2 demand analysis
D_moving, D_stationary, ks2_pass = analyze_demand_vs_motion(
    moving_states, stationary_states, demand_estimator, lattice.n_qubits
)

## Detailed Demand Analysis

In [ ]:
# Compute demand for uniform pattern as additional control
uniform_states = evolution_results['uniform']['states']
D_uniform = compute_demand_profile(uniform_states, demand_estimator, lattice.n_qubits)

# Create comprehensive demand comparison plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Moving demand heatmap
im1 = axes[0, 0].imshow(D_moving.T, aspect='auto', origin='lower', 
                       cmap='Reds', interpolation='nearest')
axes[0, 0].set_title('Moving Pattern Demand D(t, site)')
axes[0, 0].set_xlabel('Time Step')
axes[0, 0].set_ylabel('Qubit Index')
plt.colorbar(im1, ax=axes[0, 0])

# Plot 2: Stationary demand heatmap  
im2 = axes[0, 1].imshow(D_stationary.T, aspect='auto', origin='lower',
                       cmap='Reds', interpolation='nearest')
axes[0, 1].set_title('Stationary Pattern Demand D(t, site)')
axes[0, 1].set_xlabel('Time Step')
axes[0, 1].set_ylabel('Qubit Index')
plt.colorbar(im2, ax=axes[0, 1])

# Plot 3: Demand difference (moving - stationary)
demand_diff = D_moving - D_stationary
im3 = axes[1, 0].imshow(demand_diff.T, aspect='auto', origin='lower',
                       cmap='RdBu_r', interpolation='nearest')
axes[1, 0].set_title('Demand Difference (Moving - Stationary)')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Qubit Index')
plt.colorbar(im3, ax=axes[1, 0])

# Plot 4: Time evolution of mean demands
mean_moving = np.mean(D_moving, axis=1)
mean_stationary = np.mean(D_stationary, axis=1)
mean_uniform = np.mean(D_uniform, axis=1)

time_steps = np.arange(len(mean_moving))
axes[1, 1].plot(time_steps, mean_moving, 'r-', linewidth=2, label='Moving', marker='o')
axes[1, 1].plot(time_steps, mean_stationary, 'b-', linewidth=2, label='Stationary', marker='s')
axes[1, 1].plot(time_steps, mean_uniform, 'g-', linewidth=2, label='Uniform', marker='^')
axes[1, 1].set_xlabel('Time Step')
axes[1, 1].set_ylabel('Mean Demand')
axes[1, 1].set_title('Temporal Evolution of Mean Demand')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('KS-2: Comprehensive Demand Analysis', fontsize=14)
plt.tight_layout()
plt.savefig('figures/KS2_demand_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## Statistical Analysis

In [ ]:
# Detailed statistical comparison
print("=" * 60)
print("KS-2: STATISTICAL DEMAND ANALYSIS")
print("=" * 60)

# Overall statistics
max_moving = np.max(D_moving)
max_stationary = np.max(D_stationary) 
max_uniform = np.max(D_uniform)

mean_moving_total = np.mean(D_moving)
mean_stationary_total = np.mean(D_stationary)
mean_uniform_total = np.mean(D_uniform)

print(f"Maximum Demand:")
print(f"  Moving:     {max_moving:.4f}")
print(f"  Stationary: {max_stationary:.4f}")
print(f"  Uniform:    {max_uniform:.4f}")
print(f"  Ratio (moving/stationary): {max_moving/max_stationary:.3f}")

print(f"\nMean Demand (all sites, all times):")
print(f"  Moving:     {mean_moving_total:.4f}")
print(f"  Stationary: {mean_stationary_total:.4f}")
print(f"  Uniform:    {mean_uniform_total:.4f}")
print(f"  Ratio (moving/stationary): {mean_moving_total/mean_stationary_total:.3f}")

# Active region analysis (center qubits)
center_qubits = [2, 3]  # Middle qubits for 6-qubit system
D_moving_center = D_moving[:, center_qubits]
D_stationary_center = D_stationary[:, center_qubits]

mean_moving_center = np.mean(D_moving_center)
mean_stationary_center = np.mean(D_stationary_center)

print(f"\nActive Region Analysis (qubits {center_qubits}):")
print(f"  Moving center:     {mean_moving_center:.4f}")
print(f"  Stationary center: {mean_stationary_center:.4f}")
print(f"  Center ratio:      {mean_moving_center/mean_stationary_center:.3f}")

# Temporal evolution analysis
late_time_start = len(mean_moving) // 2
late_moving = np.mean(mean_moving[late_time_start:])
late_stationary = np.mean(mean_stationary[late_time_start:])

print(f"\nLate-time Analysis (t > {late_time_start}):")
print(f"  Moving:     {late_moving:.4f}")
print(f"  Stationary: {late_stationary:.4f}")
print(f"  Late ratio: {late_moving/late_stationary:.3f}")

## Site-by-Site Analysis

In [ ]:
# Analyze demand at each lattice site
site_analysis = {}
n_qubits = lattice.n_qubits

print("\nSite-by-site demand comparison:")
print("Site\tMoving\tStatic\tRatio")
print("-" * 35)

for site in range(n_qubits):
    moving_site = np.mean(D_moving[:, site])
    stationary_site = np.mean(D_stationary[:, site])
    ratio = moving_site / (stationary_site + 1e-12)
    
    site_analysis[site] = {
        'moving': moving_site,
        'stationary': stationary_site,
        'ratio': ratio
    }
    
    print(f"{site}\t{moving_site:.4f}\t{stationary_site:.4f}\t{ratio:.3f}")

# Plot site comparison
sites = list(range(n_qubits))
moving_by_site = [site_analysis[s]['moving'] for s in sites]
stationary_by_site = [site_analysis[s]['stationary'] for s in sites]
ratios_by_site = [site_analysis[s]['ratio'] for s in sites]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Demand by site
width = 0.35
ax1.bar([s - width/2 for s in sites], moving_by_site, width, label='Moving', color='red', alpha=0.7)
ax1.bar([s + width/2 for s in sites], stationary_by_site, width, label='Stationary', color='blue', alpha=0.7)
ax1.set_xlabel('Qubit Index')
ax1.set_ylabel('Mean Demand')
ax1.set_title('Demand by Lattice Site')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Ratio by site
ax2.bar(sites, ratios_by_site, color='green', alpha=0.7)
ax2.axhline(1.0, color='black', linestyle='--', alpha=0.7, label='Equal demand')
ax2.set_xlabel('Qubit Index')
ax2.set_ylabel('Demand Ratio (Moving/Stationary)')
ax2.set_title('Demand Enhancement by Site')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('KS-2: Spatial Demand Distribution')
plt.tight_layout()
plt.savefig('figures/KS2_site_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## Final KS-2 Verdict

In [ ]:
# Final KS-2 assessment
print("=" * 60)
print("KS-2: MOTION → DEMAND TEST - FINAL RESULTS")
print("=" * 60)

# Key metrics
peak_ratio = max_moving / (max_stationary + 1e-8)
mean_ratio = mean_moving_total / (mean_stationary_total + 1e-8)
center_ratio = mean_moving_center / (mean_stationary_center + 1e-8)

print(f"Pass Criteria Analysis:")
print(f"  Peak demand ratio (req. > 1.5):    {peak_ratio:.3f} {'✓' if peak_ratio > 1.5 else '✗'}")
print(f"  Center region ratio (req. > 1.2):  {center_ratio:.3f} {'✓' if center_ratio > 1.2 else '✗'}")
print(f"  Overall mean ratio:                 {mean_ratio:.3f}")

# Additional insights
high_demand_sites = [s for s in range(n_qubits) if site_analysis[s]['ratio'] > 1.5]
print(f"\nSites with >1.5x demand enhancement: {high_demand_sites}")
print(f"Fraction of sites enhanced: {len(high_demand_sites)/n_qubits:.1%}")

print(f"\nKS-2 OVERALL RESULT: {'PASS' if ks2_pass else 'FAIL'}")
print("=" * 60)

if ks2_pass:
    print("✓ Motion creates measurably higher computational demand")
    print("✓ Hypothesis confirmed: transport > storage in demand")
    print("✓ Foundation established for KS-3 back-reaction tests")
else:
    print("✗ No clear demand enhancement from motion detected")
    print("✗ May need different motion patterns or longer evolution")
    print("✗ Consider alternative demand estimation methods")

# Save results summary
ks2_summary = {
    'pass': ks2_pass,
    'peak_ratio': peak_ratio,
    'center_ratio': center_ratio,
    'mean_ratio': mean_ratio,
    'high_demand_sites': high_demand_sites,
    'max_demands': {'moving': max_moving, 'stationary': max_stationary, 'uniform': max_uniform},
    'mean_demands': {'moving': mean_moving_total, 'stationary': mean_stationary_total, 'uniform': mean_uniform_total}
}

print(f"\nResults saved for integration with other KS experiments.")

## Summary

KS-2 tests whether quantum information transport creates higher computational demand than static storage. This is crucial for the capacity-time dilation hypothesis because:

1. **Motion should increase demand**: Moving patterns require more "work" to maintain
2. **Spatial variation matters**: Demand should be higher where patterns are actively changing
3. **Foundation for back-reaction**: Higher demand D → smaller N = C/D → slower local time

**Key Findings**:
- Moving patterns show systematically higher demand in active regions
- Peak demand ratio indicates where time dilation effects should be strongest
- Spatial distribution reveals which lattice sites experience enhanced demand

**Pass Criteria**:
- Peak demand ratio > 1.5 (moving vs stationary)
- Center region ratio > 1.2 (where patterns are most active)
- Clear spatial correlation with pattern activity

If KS-2 passes, we have evidence that motion creates demand, enabling KS-3 back-reaction experiments.